In [ ]:
!pip install --upgrade 'qiskit[visualization]'

In [10]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

import numpy as np
from docplex.mp.model import Model
from math import log

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

__init__.discover_credentials:INFO:2021-03-25 16:38:31,958: Using credentials from qiskitrc
ibmqfactory.load_account:WARNING:2021-03-25 16:38:32,534: Credentials are already in use. The existing account in the session will be replaced.


# Portfolio Optimization Playground 


In [2]:
# Quantum Aggregator Wrapper

# Import algorithms 
import grovers_search as grovers
import optim_wrapper as optimization


def aggregator(algorithm, dict_details):
    if algorithm == 'grovers':
        result = grovers.grovers_search(dict_details)
    if algorithm == 'optimizer':
        result = optimization.optimize_portfolio(dict_details)
    return result


## Basic Binary Model

$\begin{aligned}
\min_{x \in \{0, 1\}^n}  q x^T \Sigma x - \mu^T x\\
\text{subject to: } 1^T x = B
\end{aligned}$

where we use the following notation:

- $x \in \{0, 1\}^n$ denotes the vector of binary decision variables, which indicate which assets to pick ($x[i] = 1$) and which not to pick ($x[i] = 0$),
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets,
- $\Sigma \in \mathbb{R}^{n \times n}$ specifies the covariances between the assets,
- $q > 0$ controls the risk appetite of the decision maker,
- and $B$ denotes the budget, i.e. the number of assets to be selected out of $n$.

We assume the following simplifications:
- all assets have the same price (normalized to 1),
- the full budget $B$ has to be spent, i.e. one has to select exactly $B$ assets.


In [3]:
# prepare problem instance
n = 6            # number of assets
q = 0.5          # risk factor
budget = n // 2  # budget
penalty = 2*n    # scaling of penalty term
mu = np.array([0.7313, 0.9893, 0.2725, 0.8750, 0.7667, 0.3622])
sigma = np.array([
    [ 0.7312, -0.6233,  0.4689, -0.5452, -0.0082, -0.3809],
    [-0.6233,  2.4732, -0.7538,  2.4659, -0.0733,  0.8945],
    [ 0.4689, -0.7538,  1.1543, -1.4095,  0.0007, -0.4301],
    [-0.5452,  2.4659, -1.4095,  3.5067,  0.2012,  1.0922],
    [-0.0082, -0.0733,  0.0007,  0.2012,  0.6231,  0.1509],
    [-0.3809,  0.8945, -0.4301,  1.0922,  0.1509,  0.8992]
])

# create docplex model
mdl = Model('portfolio_optimization')
x = mdl.binary_var_list('x{}'.format(i) for i in range(n))
objective = mdl.sum([mu[i]*x[i] for i in range(n)])
objective -= q * mdl.sum([sigma[i,j]*x[i]*x[j] for i in range(n) for j in range(n)])
mdl.maximize(objective)
mdl.add_constraint(mdl.sum(x[i] for i in range(n)) == budget)

# import the classical optimizer
from qiskit.aqua.components.optimizers import COBYLA

optim_dict = {
  "docplex_mod": mdl,
  "quantum_instance": 'qasm_simulator',
  "shots": 1024,
  "print": True,
  "solver":'vqe',
  "optimizer":COBYLA,
  "maxiter":100,
  "depth":1,
  "alpha":0.35,
  "penalty": 2*n
}

In [4]:
# Call the aggregator with 'optimizer' as the algorithm of choice
results = aggregator('optimizer', optim_dict)

014 (ms)
2021-03-25 16:33:17,871:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:INFO: Energy evaluation returned [-19.37114721] - 80.25813 (ms), eval count: 27
2021-03-25 16:33:17,873:qiskit.aqua.operators.converters.circuit_sampler:DEBUG: Parameter conversion 1.25599 (ms)
2021-03-25 16:33:17,985:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:INFO: Energy evaluation returned [-19.13732416] - 113.73520 (ms), eval count: 28
2021-03-25 16:33:18,078:qiskit.aqua.operators.converters.circuit_sampler:DEBUG: Parameter conversion 91.22276 (ms)
2021-03-25 16:33:18,169:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:INFO: Energy evaluation returned [-19.405507] - 182.82604 (ms), eval count: 29
2021-03-25 16:33:18,171:qiskit.aqua.operators.converters.circuit_sampler:DEBUG: Parameter conversion 1.15776 (ms)
2021-03-25 16:33:18,258:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:INFO: Energy evaluation returned [-19.49952494] - 88.07206 (ms), eval count: 30
2021-03-25 16:33:18,260:qiskit.aqua.

In [5]:
print(results)

{'computational_time': 7.785595865000005, 'result': optimal function value: 1.2783500000000068
optimal value: [1. 1. 0. 0. 1. 0.]
status: SUCCESS}


## Integer amount of stoks with prices, not need to invest all the budget

(Based on H1 Hypothesis in paper v1)

$\begin{aligned}
\min_{x \in \mathbb{N}^n}  q x^T \Sigma x - \mu^T x\\
\text{subject to: } S^T x \le B, x \in \mathbb{N}^n
\end{aligned}$

where we use the following notation:

- $x \in \mathbb{N}^n$ denotes the vector of integer decision variables, which indicate which assets to pick and how much ($x[i] > 0$) and which not to pick ($x[i] = 0$),
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets,
- $\Sigma \in \mathbb{R}^{n \times n}$ specifies the covariances between the assets,
- $q > 0$ controls the risk appetite of the decision maker,
- $S$ defined as the prices related to the stock,
- and $B$ denotes the budget, i.e. the maximum amount of money to be spent purchasing assets.

In [6]:
# prepare problem instance
n = 6            # number of assets
q = 0.5          # risk factor
budget = 100  # budget
penalty = 2*n    # scaling of penalty term

# example instance
s = np.array([1, 2, 3, 5, 8, 13])
mu = np.array([0.7313, 0.9893, 0.2725, 0.8750, 0.7667, 0.3622])
sigma = np.array([
    [ 0.7312, -0.6233,  0.4689, -0.5452, -0.0082, -0.3809],
    [-0.6233,  2.4732, -0.7538,  2.4659, -0.0733,  0.8945],
    [ 0.4689, -0.7538,  1.1543, -1.4095,  0.0007, -0.4301],
    [-0.5452,  2.4659, -1.4095,  3.5067,  0.2012,  1.0922],
    [-0.0082, -0.0733,  0.0007,  0.2012,  0.6231,  0.1509],
    [-0.3809,  0.8945, -0.4301,  1.0922,  0.1509,  0.8992]
])


# create docplex model
mdl = Model('portfolio_optimization')
x = mdl.integer_var_list('x{}'.format(i) for i in range(n))
objective = mdl.sum([mu[i]*x[i] for i in range(n)])
objective -= q * mdl.sum([sigma[i,j]*x[i]*x[j] for i in range(n) for j in range(n)])
mdl.maximize(objective)
mdl.add_constraint(mdl.sum(s[i]*x[i] for i in range(n)) <= budget)

# import the classical optimizer
from qiskit.aqua.components.optimizers import COBYLA

optim_dict = {
  "docplex_mod": mdl,
  "quantum_instance": 'qasm_simulator',
  "shots": 1024,
  "print": True,
  "solver":'vqe',
  "optimizer":COBYLA,
  "maxiter":100,
  "depth":1,
  "alpha":0.35,
  "penalty": 2*n
}

In [7]:
# Call the aggregator with 'optimizer' as the algorithm of choice
results = aggregator('optimizer', optim_dict)

int_slack@0
      - 175640385467449344 x5@47*c0@int_slack@1
      - 351280770934898688 x5@47*c0@int_slack@2
      - 702561541869797376 x5@47*c0@int_slack@3
      - 1405123083739594752 x5@47*c0@int_slack@4
      - 2810246167479189504 x5@47*c0@int_slack@5
      - 3249347131147812864 x5@47*c0@int_slack@6 - 1e+20 x5@48^2
      - 1e+20 x5@48*x5@49 - 1e+20 x5@48*x5@50 - 1e+20 x5@48*x5@51
      - 1e+20 x5@48*x5@52 - 1e+20 x5@48*x5@53 - 1e+20 x5@48*x5@54
      - 1e+20 x5@48*x5@55 - 1e+20 x5@48*x5@56 - 1e+20 x5@48*x5@57
      - 1e+20 x5@48*x5@58 - 1e+20 x5@48*x5@59 - 1e+20 x5@48*x5@60
      - 1e+20 x5@48*x5@61 - 1e+20 x5@48*x5@62 - 1e+20 x5@48*x5@63
      - 1e+20 x5@48*x5@64 - 1e+20 x5@48*x5@65 - 1e+20 x5@48*x5@66
      - 175640385467449344 x5@48*c0@int_slack@0
      - 351280770934898688 x5@48*c0@int_slack@1
      - 702561541869797376 x5@48*c0@int_slack@2
      - 1405123083739594752 x5@48*c0@int_slack@3
      - 2810246167479189504 x5@48*c0@int_slack@4
      - 5620492334958379008 x5@48*c0@int_sl

CircuitError: "Register size must be positive (int '0' was provided)"

## Binary representation of integers limited to budget amount

(Based on H2 Hypothesis in paper v1)

$\begin{aligned}
\min_{x \in \mathbb{N}^n}  q x^T \Sigma x - \mu^T x\\
\end{aligned}$

subject to:

$\begin{aligned}
\ S^T x \le B, x \in \mathbb{N}^n\\
\ x = \sum_{i=0}^{log_{2}B}2^iy_i, y \in \{0,1\}
\end{aligned}$

where we use the following notation:

- $x \in \mathbb{N}^n$ denotes the vector of integer decision variables, which indicate which assets to pick and how much ($x[i] > 0$) and which not to pick ($x[i] = 0$),
- $y$ represents the binary decomposition of $x$
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets,
- $\Sigma \in \mathbb{R}^{n \times n}$ specifies the covariances between the assets,
- $q > 0$ controls the risk appetite of the decision maker,
- $S$ defined as the prices related to the stock,
- and $B$ denotes the budget, i.e. the maximum amount of money to be spent purchasing assets.


In [8]:
# prepare problem instance
n = 3            # number of assets
q = 0.5          # risk factor
budget = 100  # budget
penalty = budget*n    # scaling of penalty term

# example instance
s = np.array([1, 3, 8])
mu = np.array([0.7313, 0.9893, 0.2725])
sigma = np.array([
    [ 0.7312, -0.6233,  0.4689],
    [-0.6233,  2.4732, -0.7538],
    [ 0.4689, -0.7538,  1.1543]
])

In [12]:
# create docplex model
mdl = Model('portfolio_optimization')

budget_bits = int(log(budget,2))

y_var = mdl.binary_var_list('y{}_{}'.format(i, j) for i in range(n) for j in range(budget_bits))

y = np.zeros([n, budget_bits])
y = y.astype('object')

var_location = np.zeros([n, budget_bits])
var_location = var_location == 0

y[var_location] = y_var

#x = (mdl.sum([(2**k)*y[i][k] for k in range(budget_bits)]) for i in range(n))

objective = mdl.sum([mu[i]*mdl.sum([(2**k)*y[i][k] for k in range(budget_bits)]) for i in range(n)])
objective -= q * mdl.sum([sigma[i,j]*mdl.sum([(2**k)*y[i][k] for k in range(budget_bits)]) * mdl.sum([(2**k)*y[j][k] for k in range(budget_bits)]) for i in range(n) for j in range(n)])
mdl.maximize(objective)
mdl.add_constraint(mdl.sum(s[i]*mdl.sum([(2**k)*y[i][k] for k in range(budget_bits)]) for i in range(n)) <= budget)

# import the classical optimizer
from qiskit.aqua.components.optimizers import COBYLA

optim_dict = {
  "docplex_mod": mdl,
  "quantum_instance": 'qasm_simulator',
  "shots": 1024,
  "print": True,
  "solver":'vqe',
  "optimizer":COBYLA,
  "maxiter":100,
  "depth":1,
  "alpha":0.35,
  "penalty": 2*n
}

In [16]:
# Call the aggregator with 'optimizer' as the algorithm of choice
results = aggregator('optimizer', optim_dict)

2021-03-26 21:20:51,349:qiskit.aqua.quantum_instance:INFO: 
Qiskit Terra version: 0.16.4
Backend: 'qasm_simulator (AerProvider)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cu1', 'cu2', 'cu3', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'h', 'id', 'initialize', 'kraus', 'mcp', 'mcr', 'mcrx', 'mcry', 'mcrz', 'mcswap', 'mcsx', 'mcu1', 'mcu2', 'mcu3', 'mcx', 'mcy', 'mcz', 'multiplexer', 'p', 'r', 'roerror', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'snapshot', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z'], 'coupling_map': None}
{'initial_layout': None, 'seed_transpiler': None, 'optimization_level': None}
RunConfig(max_credits=10, shots=1024)
{'timeout': None}
{}
{}
Measurement mitigation: None
### Original problem:
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: portfolio_optimization

Maximize
 obj: 1.462600000000 y0_0 + 2.193900000000 y0_1 + 0.731300000000 y0_3
      + 4.387800000000 y0_4 

In [14]:
results['result'].variables_dict

{'y0_0': 0.0,
 'y0_1': 0.0,
 'y0_2': 1.0,
 'y0_3': 0.0,
 'y0_4': 0.0,
 'y0_5': 0.0,
 'y1_0': 1.0,
 'y1_1': 1.0,
 'y1_2': 0.0,
 'y1_3': 0.0,
 'y1_4': 0.0,
 'y1_5': 0.0,
 'y2_0': 0.0,
 'y2_1': 0.0,
 'y2_2': 0.0,
 'y2_3': 1.0,
 'y2_4': 1.0,
 'y2_5': 0.0,
 'c0@int_slack@0': 1.0,
 'c0@int_slack@1': 0.0,
 'c0@int_slack@2': 1.0,
 'c0@int_slack@3': 1.0,
 'c0@int_slack@4': 1.0,
 'c0@int_slack@5': 0.0,
 'c0@int_slack@6': 0.0}

In [15]:
print('Binary variables')
ms = np.array(list(results['result'].variables_dict.values())).astype(int)
print(ms)

print()
print('Binary results(reverse order): y')
y_val= np.zeros([n, budget_bits]).astype(int)
y_val=[[ms[i+j*budget_bits] for i in range(budget_bits)] for j in range(n)]
print(y_val)

print()
print('Integer results (amount of stocks): x')
x_val= np.zeros(n).astype(int)
for i, a in enumerate(y_val):
    a.reverse()
    x_val[i] = int("".join(str(y) for y in a), 2)
print(x_val)

print()
print('Amount invest in each stock:')
invest = s*x_val
print(invest)

Binary variables
[0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 0]

Binary results(reverse order): y
[[0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0]]

Integer results (amount of stocks): x
[ 4  3 24]

Amount invest in each stock:
[  4   9 192]


In [7]:
import qiskit.tools.jupyter
%qiskit_version_table

In [8]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.16.4',
 'qiskit-aer': '0.7.6',
 'qiskit-ignis': '0.5.2',
 'qiskit-ibmq-provider': '0.12.1',
 'qiskit-aqua': '0.8.2',
 'qiskit': '0.24.0'}